In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
datasets = [
    ["abalone_input.pd","abalone_target.pd"], 
    ["adult_input.pd","adult_target.pd"], # Large
    ["banknote_input.pd","banknote_target.pd"], # Easy
    ["bank_input.pd","bank_target.pd"], # Large
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["cars_input.pd","cars_target.pd"], 
    ["contraceptive_input.pd","contraceptive_target.pd"], 
    ["generated6_input.np","generated6_target.np"],
    ["hrss_input.pd","hrss_target.pd"], # Large
    ["iris_input.pd","iris_target.pd"],
    ["steel_input.pd","steel_target.pd"],
    ["students_input.pd","students_target.pd"],
    #["sensorless_input.pd","sensorless_target.pd"], # Very Large dataset
]

In [3]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": 3,
    "iterations": 200,
    "max_depth": 2,
    "init_learning_rate": 10,
    "learning_rate_decay": 0.985,
    "initialization_method": "to be overwritten",
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": "to be overwritten",
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    }
parameters_fit = {
    "optimization_method": "least_squares_linear_regression",
    "early_stopping": False,
    }

In [4]:
gate_reduction_methods = [
    "feature_importance_lda",
    None
]

In [5]:
initialization_methods = [
    Random_init(seed=None),
    Kmeans_init(),
    KDTmeans_init(alpha=2.5, beta=0.15, gamma=0.12),
    BGM_init(n_components=parameters["n_experts"],
             mean_precision_prior=0.6,
             weight_concentration_prior_type="dirichlet_distribution",
             weight_concentration_prior=0.3,
             weight_cutoff=0.0)    
]

In [6]:
start = timer()
runs = 10
rows = []
for dataset in datasets:
    print("Starting",dataset[0],"...")
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    
    use_dataframe = False
    if isinstance(data_input, pd.core.frame.DataFrame):
        use_dataframe = True
        
    dict_results = {
        "dataset" : dataset[0],
        "n_features" : data_input.shape[1]
    } 
    
    for method in gate_reduction_methods:
        for init_method in initialization_methods:
            init_method_name = init_method.__class__.__name__
            
            print("Starting",dataset[0],method,init_method_name,"...")
            parameters["use_2_dim_gate_based_on"] = method
            if parameters["use_2_dim_gate_based_on"] is not None:
                parameters["use_2_dim_clustering"] = True
            else:
                parameters["use_2_dim_clustering"] = False
            parameters["initialization_method"] = init_method

            train_accuracies = []
            val_accuracies = []
            train_accuracies_i = [[] for i in range(parameters["iterations"])]
            
            rkf = RepeatedKFold(n_splits=4, n_repeats=runs)
            for train_idx, val_idx in rkf.split(data_input):
                if use_dataframe:
                    X_temp = data_input.iloc[train_idx]
                    y_temp = data_target.iloc[train_idx]
                    X_temp.reset_index(inplace=True, drop=True)
                    y_temp.reset_index(inplace=True, drop=True)
                else:
                    X_temp = data_input[train_idx]
                    y_temp = data_target[train_idx]

                parameters["X"] = X_temp
                parameters["y"] = y_temp

                modt = MoDT(**parameters)
                modt.fit(**parameters_fit)
                train_accuracies.append(modt.score_internal_disjoint())

                if use_dataframe:
                    X_temp = data_input.iloc[val_idx]
                    y_temp = data_target.iloc[val_idx]
                    X_temp.reset_index(inplace=True, drop=True)
                    y_temp.reset_index(inplace=True, drop=True)
                else:
                    X_temp = data_input[val_idx]
                    y_temp = data_target[val_idx]
                val_accuracies.append(modt.score(X_temp, y_temp))
                for i in range(parameters["iterations"]):
                    train_accuracies_i[i].append(modt.all_accuracies[i])

            train_accuracy = np.mean(train_accuracies)
            val_accuracy = np.mean(val_accuracies)
            train_std = np.std(train_accuracies)
            val_std = np.std(val_accuracies)
            dict_results[str(method) + "_" + str(init_method_name) + "_train"] = train_accuracy
            dict_results[str(method) + "_" + str(init_method_name) + "_test"] = val_accuracy
            dict_results[str(method) + "_" + str(init_method_name) + "_train_std"] = train_std
            dict_results[str(method) + "_" + str(init_method_name) + "_test_std"] = val_std                        

            for i in range(parameters["iterations"]):
                train_i_acc = np.mean(train_accuracies_i[i])
                dict_results[str(method) + "_" + str(init_method_name) + "_train_i_" + str(i)] = modt.all_accuracies[i]
        
    rows.append(dict_results)
    
print("Duration", timer() - start)
df_performance = pd.DataFrame(rows)

Starting abalone_input.pd ...
Starting abalone_input.pd feature_importance_lr_max Random_init ...
Starting abalone_input.pd feature_importance_lr_max Kmeans_init ...
Starting abalone_input.pd feature_importance_lr_max KDTmeans_init ...
Starting abalone_input.pd feature_importance_lr_max BGM_init ...
Starting abalone_input.pd None Random_init ...
Starting abalone_input.pd None Kmeans_init ...
Starting abalone_input.pd None KDTmeans_init ...
Starting abalone_input.pd None BGM_init ...
Starting adult_input.pd ...
Starting adult_input.pd feature_importance_lr_max Random_init ...
Starting adult_input.pd feature_importance_lr_max Kmeans_init ...
Starting adult_input.pd feature_importance_lr_max KDTmeans_init ...
Starting adult_input.pd feature_importance_lr_max BGM_init ...
Starting adult_input.pd None Random_init ...
Starting adult_input.pd None Kmeans_init ...
Starting adult_input.pd None KDTmeans_init ...
Starting adult_input.pd None BGM_init ...


C:\Users\Robert\Anaconda3\envs\MA1\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


Starting banknote_input.pd ...
Starting banknote_input.pd feature_importance_lr_max Random_init ...
Starting banknote_input.pd feature_importance_lr_max Kmeans_init ...
Starting banknote_input.pd feature_importance_lr_max KDTmeans_init ...
Starting banknote_input.pd feature_importance_lr_max BGM_init ...
Starting banknote_input.pd None Random_init ...
Starting banknote_input.pd None Kmeans_init ...
Starting banknote_input.pd None KDTmeans_init ...
Starting banknote_input.pd None BGM_init ...
Starting bank_input.pd ...
Starting bank_input.pd feature_importance_lr_max Random_init ...
Starting bank_input.pd feature_importance_lr_max Kmeans_init ...
Starting bank_input.pd feature_importance_lr_max KDTmeans_init ...
Starting bank_input.pd feature_importance_lr_max BGM_init ...
Starting bank_input.pd None Random_init ...
Starting bank_input.pd None Kmeans_init ...
Starting bank_input.pd None KDTmeans_init ...
Starting bank_input.pd None BGM_init ...


C:\Users\Robert\Anaconda3\envs\MA1\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


Starting breast_cancer_input.np ...
Starting breast_cancer_input.np feature_importance_lr_max Random_init ...
Starting breast_cancer_input.np feature_importance_lr_max Kmeans_init ...
Starting breast_cancer_input.np feature_importance_lr_max KDTmeans_init ...
Starting breast_cancer_input.np feature_importance_lr_max BGM_init ...


...

C:\Users\Robert\Anaconda3\envs\MA1\lib\site-packages\sklearn\discriminant_analysis.py:480: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = (S**2 / np.sum(


<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init o

C:\Users\Robert\Anaconda3\envs\MA1\lib\site-packages\sklearn\discriminant_analysis.py:480: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = (S**2 / np.sum(


<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init o

<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init object at 0x00000298EEEABF40> separation unsuccessful. Gate initialized randomly instead.
<modt._initialization.BGM_init o

In [7]:
df_performance

,dataset,n_features,feature_importance_lr_max_Random_init_train,feature_importance_lr_max_Random_init_test,feature_importance_lr_max_Random_init_train_std,feature_importance_lr_max_Random_init_test_std,feature_importance_lr_max_Random_init_train_i_0,feature_importance_lr_max_Random_init_train_i_1,feature_importance_lr_max_Random_init_train_i_2,feature_importance_lr_max_Random_init_train_i_3,...,None_BGM_init_train_i_190,None_BGM_init_train_i_191,None_BGM_init_train_i_192,None_BGM_init_train_i_193,None_BGM_init_train_i_194,None_BGM_init_train_i_195,None_BGM_init_train_i_196,None_BGM_init_train_i_197,None_BGM_init_train_i_198,None_BGM_init_train_i_199
0,abalone_input.pd,8,0.729463,0.717692,0.006548,0.012734,0.676987,0.680817,0.684966,0.686882,...,0.750080,0.750080,0.750080,0.750080,0.750080,0.750080,0.750080,0.750080,0.750080,0.750080
1,adult_input.pd,14,0.837719,0.837524,0.005670,0.005849,0.807975,0.808019,0.807665,0.806914,...,0.825745,0.825745,0.825745,0.825745,0.825745,0.825745,0.825745,0.825745,0.825745,0.825745
2,banknote_input.pd,4,0.973955,0.966545,0.009557,0.011734,0.910593,0.915452,0.924198,0.931973,...,0.987366,0.987366,0.987366,0.987366,0.987366,0.987366,0.987366,0.987366,0.987366,0.987366
3,bank_input.pd,20,0.908325,0.906754,0.002031,0.002715,0.897413,0.897511,0.897640,0.897478,...,0.911139,0.911139,0.911139,0.911139,0.911139,0.911139,0.911139,0.911139,0.911139,0.911139
4,breast_cancer_input.np,10,0.963800,0.924783,0.008284,0.018329,0.939110,0.939110,0.939110,0.943794,...,0.969555,0.969555,0.969555,0.969555,0.969555,0.969555,0.969555,0.969555,0.969555,0.969555
5,cars_input.pd,6,0.792284,0.789815,0.015256,0.019271,0.778549,0.778549,0.778549,0.778549,...,0.897377,0.898920,0.906636,0.905864,0.904321,0.907407,0.898148,0.898148,0.902778,0.902778
6,contraceptive_input.pd,9,0.569268,0.541616,0.015834,0.021167,0.516742,0.516742,0.516742,0.516742,...,0.505882,0.505882,0.505882,0.505882,0.505882,0.505882,0.505882,0.505882,0.505882,0.505882
7,generated6_input.np,2,0.872160,0.866180,0.031048,0.033697,0.741333,0.739733,0.739733,0.739733,...,0.858933,0.858933,0.858933,0.858933,0.858933,0.858933,0.858933,0.858933,0.858933,0.858933
8,hrss_input.pd,18,0.767395,0.767207,0.002251,0.004301,0.764069,0.764069,0.764069,0.764069,...,0.780647,0.780647,0.780647,0.780647,0.780647,0.780647,0.780647,0.780647,0.780647,0.780647
9,iris_input.pd,4,0.986447,0.947973,0.007148,0.027396,0.964602,0.982301,0.982301,0.982301,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
pickle.dump(df_performance, open("dataframes/ex3_df_initialization_methods.pd", "wb"))